# Histograms for the obtained data.
- We have a .npy file with each column representing a different time instance having 8 million data points of VonMises stress. We are interested in constructing a histogram and see its evolutionfor these von mises stresses.

In [20]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.preprocessing import MinMaxScaler

# Code to create a directories
o_folder = "/Users/Jayee/Library/CloudStorage/OneDrive-UW-Madison/PhD(my_PC)/\
PhD(output_files)/Engineering_projects/Shear_loading/240709_hist/abs_rotat"
ylim = 0.015
data_number_list = [700,800,801,802]
quantity_list = ["e1","e2","e3"]

for quantity in quantity_list:
    for save_file_number in data_number_list:
        organization1 = os.path.join(quantity,str(save_file_number),"global_scaling")
        output_folder = os.path.join(o_folder,organization1)
        matplotlib.use('Agg')
        # quantity = "e2"
        # save_file_number = "700"
        # output_folder = f"/Users/Jayee/Library/CloudStorage/OneDrive-UW-Madison/PhD(my_PC)/\
        # PhD(output_files)/Engineering_projects/Shear_loading/240709_hist/\
        # {quantity}/{save_file_number}/global_scaling"

        for file_index in range(11):
            file_index = file_index +1
            file_number = f"{file_index}"
            # location of the numpy file
            if quantity == "vm_stress":
                file_number = f"{file_index:03}"
            npy_file_loc = f"/Volumes/Extreme SSD/mac_data/shear_data/abs_rotat_data/\
{quantity}/{save_file_number}/{quantity}/Step-{file_number}.npy"

            vm_stress = np.load(npy_file_loc)

            # defining the limiting factor
            limiting_factor = 0.8
            
            if not ("global_scaling" and "rotat" in output_folder):
                # max and min elements of the numpy array
                ## This is important in determining the bins and constructing a uniform histogram.
                min_vmstress = np.min(np.min(vm_stress))
                max_vmstress = np.max(np.max(vm_stress))
                limiting_stress = limiting_factor * max_vmstress

            # Obtaining the Histogram
            if quantity == "vm_stress":
                bin_edges_dummy = np.linspace(0,1500,1500)
                #bin_edges_dummy = np.linspace(0,max_vmstress,1500)

            if quantity == "tdd":
                #bin_edges_dummy = np.linspace(min_vmstress,max_vmstress,1500)
                bin_edges_dummy = np.linspace(2.4e6,1.7e9,1500)

            if quantity == "e1" or quantity == "e2" or quantity == "e3":
                #bin_edges_dummy = np.linspace(min_vmstress,max_vmstress,1500)
                bin_edges_dummy = np.linspace(-10,+10,1500)
                if ("scaled_wrt_max_value" in output_folder):
                    bin_edges_dummy = np.linspace(min_vmstress,max_vmstress,1500)

            freq,bin_edges = np.histogram(vm_stress, bin_edges_dummy)
            cumulative_freq = np.cumsum(freq)

            rel_freq = freq/(8e6+1)
            cumulative_rel_freq = cumulative_freq/(8e6+1)

            # Finidng the indices of the limiting stress
            if not ("global_scaling" and "rotat" in output_folder):
                limiting_indices = np.delete(np.where(bin_edges>=limiting_stress)[0],-1)

                # # of elements > limiting stress
                no_of_limiting_elements = sum(freq[limiting_indices])
                percentage_of_limiting_elements = float(sum(rel_freq[limiting_indices])*100)

            # Plotting

            ## defining color bar
            values = [0, 0.5, 1]
            color_list = ["green", "yellow", "red"]
            my_color_map= mcolors.LinearSegmentedColormap.from_list("my_color_map", list(zip(values,color_list)))

            ## Defining the color values
            scaler = MinMaxScaler()
            scaler.fit(cumulative_rel_freq.reshape(-1,1))
            norm_values_color = scaler.transform(cumulative_rel_freq.reshape(-1,1))
            c_values = my_color_map(norm_values_color)


            ## Definig the axis labels and title
            fig,axs = plt.subplots(1,1,figsize=[10,8])
            axs.set_title(f"Histogram corresponding to {save_file_number}- {file_number}", fontweight="bold", fontsize=16)
            axs.set_xlabel(f"{quantity} values", fontsize=14, fontweight="bold")
            axs.set_ylabel("Relative frequency", fontweight="bold", fontsize=14)

            # Plotting the histogram
            zip_list = list(zip(rel_freq,bin_edges[:-1],bin_edges[1:]))
            for index, (count,left,right) in enumerate(zip_list):
                axs.fill_between([left,right],0,count,color=c_values[index],zorder=2)
            
            if not ("global_scaling" and "rotat" in output_folder):
                ## Plotting the maximum stress and the limiting stress
                axs.scatter(max_vmstress,0, c="Blue",s=100,edgecolor="k",linewidths=2,zorder=3,label=f"Max. {quantity}")
                if limiting_stress > min_vmstress:
                    axs.scatter(limiting_stress,0, c="brown",s=100,edgecolor="k",linewidths=2,zorder=3, label= f"80% of Max.{quantity}")
            
                ## Adding text to plot
                axs.text(limiting_stress,0.2*max(rel_freq),
                         f"# of limiting points \n \
                         = {no_of_limiting_elements}\n \
                         = {percentage_of_limiting_elements:.2}%",ha = "left",fontsize=12)
                
            if "rotat" in output_folder:
                axs.set_ylim(0, ylim)
                
            ## Defining the legend and aesthetics
            axs.legend(loc="best",fontsize=12,facecolor="pink",edgecolor="k")
            axs.set_facecolor("#f4f4f4")
            axs.grid(linestyle="--", zorder=1)
            axs.tick_params(labelsize=12)

            ## Saving the plot
            file_name = f"{file_number}.pdf"
            output_file_path = os.path.join(output_folder,file_name)
            fig.savefig(output_file_path)
            plt.close(fig) 
            print(quantity, save_file_number,file_number)

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


e1 700 1


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


e1 700 2


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


...

e3 802 9


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


e3 802 10


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


e3 802 11


In [1]:
import numpy as np
# say we want the vonmises stress at all the gauss quadrature points
# for step-n
npy_file_loc = "npy file path for step-n"
vm_stress_corresponding_to_step_n = np.load(npy_file_loc)

In [12]:
("vm" and "ma" in "max_vm")!


SyntaxError: invalid syntax (311229914.py, line 1)